In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

from pathlib import Path
from sklearn import metrics
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import torchvision

from datetime import datetime
import cv2
from collections import OrderedDict

In [3]:
import pickle

In [4]:
PATH = Path("/data2/yinterian/multi-task-romain")

In [5]:
summary = pd.read_csv(PATH/"summary_data.csv")
summary.head()

,subject_id,gender,age,sapsi_first,sofa_first,care_unit,amine,sedation,ventilation,period,...,W1_abpm,W1_abpd,W1_abps,V1_hr,V1_spo2,V1_abpm,V1_abpd,V1_abps,event,mean_abpm_prediction
0,10013,F,87,15,8,1,1,0,0,10,...,4.048186,9.050967,2.103643,472.806949,515.251034,325.923193,658.457835,200.447095,1,62.465
1,10013,F,87,15,8,1,1,0,0,11,...,-9.316132,-16.033646,-5.427045,486.577854,531.797332,328.398067,644.828351,201.154202,1,56.505
2,10013,F,87,15,8,1,1,0,0,12,...,-0.989949,1.184404,-1.202082,518.026428,16.387200,345.739861,637.368375,215.879700,1,60.520
3,10013,F,87,15,8,1,1,0,0,15,...,-14.743176,1.644023,-3.252691,504.255523,518.238560,322.369982,562.945386,205.025611,1,55.230
4,10013,F,87,15,8,1,1,0,0,16,...,-0.530330,-7.247845,1.626346,501.020510,506.147034,339.269834,632.224173,219.804143,1,57.965


In [75]:
summary["key"] = summary.apply(lambda x:'%s-%s' % (str(x["subject_id"]),str(x["period"])),axis=1)
summary.head()                               

,subject_id,gender,age,sapsi_first,sofa_first,care_unit,amine,sedation,ventilation,period,...,W1_abpd,W1_abps,V1_hr,V1_spo2,V1_abpm,V1_abpd,V1_abps,event,mean_abpm_prediction,key
0,10013,F,87,15,8,1,1,0,0,10,...,9.050967,2.103643,472.806949,515.251034,325.923193,658.457835,200.447095,1,62.465,10013-10
1,10013,F,87,15,8,1,1,0,0,11,...,-16.033646,-5.427045,486.577854,531.797332,328.398067,644.828351,201.154202,1,56.505,10013-11
2,10013,F,87,15,8,1,1,0,0,12,...,1.184404,-1.202082,518.026428,16.387200,345.739861,637.368375,215.879700,1,60.520,10013-12
3,10013,F,87,15,8,1,1,0,0,15,...,1.644023,-3.252691,504.255523,518.238560,322.369982,562.945386,205.025611,1,55.230,10013-15
4,10013,F,87,15,8,1,1,0,0,16,...,-7.247845,1.626346,501.020510,506.147034,339.269834,632.224173,219.804143,1,57.965,10013-16


In [84]:
outcome = {row["key"]: (row["event"], row["mean_abpm_prediction"]) for i,row in summary.iterrows()}

In [76]:
data = pd.read_csv(PATH/"full_data.csv")
data.head()

,subject_id,gender,age,sapsi_first,sofa_first,care_unit,amine,sedation,ventilation,period,window,time_and_date,hr,spo2,abpsys,abpdias,abpmean,event
0,10013,F,87,15,8,1,1,0,0,10,obs,2564-11-02 23:57:18,85.0,89.1,106.8,31.5,52.2,1
1,10013,F,87,15,8,1,1,0,0,10,obs,2564-11-02 23:58:18,84.9,88.2,106.2,31.5,52.1,1
2,10013,F,87,15,8,1,1,0,0,10,obs,2564-11-02 23:59:18,85.1,87.9,109.0,32.4,53.7,1
3,10013,F,87,15,8,1,1,0,0,10,obs,2564-11-03 00:00:18,84.2,88.0,107.1,31.9,52.8,1
4,10013,F,87,15,8,1,1,0,0,10,obs,2564-11-03 00:01:18,85.7,86.6,107.0,31.8,52.6,1


In [77]:
## just observations
obs = data[data["window"] == "obs"]
obs.shape

(2375280, 18)

In [78]:
obs["key"] = obs.apply(lambda x:'%s-%s' % (str(x["subject_id"]),str(x["period"])),axis=1)

/home/yinterian/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [79]:
obs.head()

,subject_id,gender,age,sapsi_first,sofa_first,care_unit,amine,sedation,ventilation,period,window,time_and_date,hr,spo2,abpsys,abpdias,abpmean,event,key
0,10013,F,87,15,8,1,1,0,0,10,obs,2564-11-02 23:57:18,85.0,89.1,106.8,31.5,52.2,1,10013-10
1,10013,F,87,15,8,1,1,0,0,10,obs,2564-11-02 23:58:18,84.9,88.2,106.2,31.5,52.1,1,10013-10
2,10013,F,87,15,8,1,1,0,0,10,obs,2564-11-02 23:59:18,85.1,87.9,109.0,32.4,53.7,1,10013-10
3,10013,F,87,15,8,1,1,0,0,10,obs,2564-11-03 00:00:18,84.2,88.0,107.1,31.9,52.8,1,10013-10
4,10013,F,87,15,8,1,1,0,0,10,obs,2564-11-03 00:01:18,85.7,86.6,107.0,31.8,52.6,1,10013-10


In [81]:
obs_periods = np.unique(obs.key.values)

In [82]:
obs_periods[:10]

array(['10013-10', '10013-11', '10013-12', '10013-15', '10013-16',
       '10013-17', '10013-18', '10013-2', '10013-20', '10013-22'],
      dtype=object)

In [87]:
period = obs[obs["key"] == obs_periods[0]]

In [88]:
period.head()

,subject_id,gender,age,sapsi_first,sofa_first,care_unit,amine,sedation,ventilation,period,window,time_and_date,hr,spo2,abpsys,abpdias,abpmean,event,key
0,10013,F,87,15,8,1,1,0,0,10,obs,2564-11-02 23:57:18,85.0,89.1,106.8,31.5,52.2,1,10013-10
1,10013,F,87,15,8,1,1,0,0,10,obs,2564-11-02 23:58:18,84.9,88.2,106.2,31.5,52.1,1,10013-10
2,10013,F,87,15,8,1,1,0,0,10,obs,2564-11-02 23:59:18,85.1,87.9,109.0,32.4,53.7,1,10013-10
3,10013,F,87,15,8,1,1,0,0,10,obs,2564-11-03 00:00:18,84.2,88.0,107.1,31.9,52.8,1,10013-10
4,10013,F,87,15,8,1,1,0,0,10,obs,2564-11-03 00:01:18,85.7,86.6,107.0,31.8,52.6,1,10013-10


In [28]:
static_cols = period.columns[12:18]
static_cols

Index(['hr', 'spo2', 'abpsys', 'abpdias', 'abpmean', 'event'], dtype='object')

In [93]:
def process_periods(df, key, outcome, index):
    y1, y2 = outcome[key]
    static_df = df.iloc[0, :10]
    time_df = df.iloc[:,12:18]
    time_series = [time_df.iloc[i,].values for i in range(time_df.shape[0])]
    static_df["series"] = time_series
    static_df["y_event"] = y1
    static_df["y_mean_abpm"] = y2
    return static_df.copy().to_frame(index).T

In [92]:
period = obs[obs["key"] == obs_periods[0]]
process_periods(period,  obs_periods[0], outcome)

,subject_id,gender,age,sapsi_first,sofa_first,care_unit,amine,sedation,ventilation,period,series,y_event,y_mean_abpm
0,10013,F,87,15,8,1,1,0,0,10,"[[85.0, 89.1, 106.8, 31.5, 52.2, 1.0], [84.9, ...",1,62.465


In [98]:
def process_whole_list(obs):
    list_df = []
    obs_periods = np.unique(obs.key.values)
    for i in range(len(obs_periods)):
        key = obs_periods[i]
        print(key)
        period = obs[obs["key"] == key]
        line = process_periods(period, key, outcome, i)
        list_df.append(line)
    return list_df

In [95]:
test = obs.iloc[:1000]
test.head()

,subject_id,gender,age,sapsi_first,sofa_first,care_unit,amine,sedation,ventilation,period,window,time_and_date,hr,spo2,abpsys,abpdias,abpmean,event,key
0,10013,F,87,15,8,1,1,0,0,10,obs,2564-11-02 23:57:18,85.0,89.1,106.8,31.5,52.2,1,10013-10
1,10013,F,87,15,8,1,1,0,0,10,obs,2564-11-02 23:58:18,84.9,88.2,106.2,31.5,52.1,1,10013-10
2,10013,F,87,15,8,1,1,0,0,10,obs,2564-11-02 23:59:18,85.1,87.9,109.0,32.4,53.7,1,10013-10
3,10013,F,87,15,8,1,1,0,0,10,obs,2564-11-03 00:00:18,84.2,88.0,107.1,31.9,52.8,1,10013-10
4,10013,F,87,15,8,1,1,0,0,10,obs,2564-11-03 00:01:18,85.7,86.6,107.0,31.8,52.6,1,10013-10


In [102]:
list_obs = process_whole_list(obs)

10013-10
10013-11
10013-12
10013-15
10013-16
10013-17
10013-18
10013-2
10013-20
10013-22
10013-23
10013-24
10013-25
10013-27
10013-28
10013-3
10013-31
10013-32
10013-33
10013-4
10013-5
10013-6
10013-7
10013-8
10013-9
1006-1
1006-10
1006-12
1006-13
1006-14
1006-15
1006-17
1006-19
1006-2
1006-24
1006-25
1006-26
1006-28
1006-29
1006-30
1006-31
1006-33
1006-34
1006-35
1006-36
1006-37
1006-38
1006-39
1006-40
1006-41
1006-42
1006-43
1006-44
1006-45
1006-46
10061-10
10061-100
10061-101
10061-102
10061-104
10061-105
10061-108
10061-109
10061-11
10061-110
10061-111
10061-112
10061-113
10061-114
10061-115
10061-116
10061-117
10061-118
10061-12
10061-120
10061-121
10061-123
10061-124
10061-125
10061-126
10061-128
10061-13
10061-134
10061-135
10061-136
10061-137
10061-138
10061-139
10061-142
10061-144
10061-145
10061-146
10061-147
10061-148
10061-149
10061-15
10061-150
10061-151
10061-153
10061-154
10061-155
10061-157
10061-158
10061-159
10061-16
10061-160
10061-161
10061-162
10061-163
10061-164
1

11066-75
11066-76
11066-79
11066-80
11066-81
11066-82
11066-83
11066-84
11066-85
11066-87
11096-1
11096-10
11096-11
11096-12
11096-13
11096-14
11096-18
11096-19
11096-2
11096-20
11096-21
11096-23
11096-27
11096-28
11096-3
11096-4
11096-6
11096-9
11099-1
11099-10
11099-100
11099-101
11099-102
11099-103
11099-104
11099-106
11099-107
11099-108
11099-109
11099-11
11099-110
11099-111
11099-114
11099-115
11099-116
11099-117
11099-118
11099-12
11099-120
11099-121
11099-122
11099-123
11099-124
11099-125
11099-126
11099-127
11099-128
11099-129
11099-13
11099-130
11099-131
11099-132
11099-133
11099-134
11099-135
11099-136
11099-137
11099-138
11099-139
11099-14
11099-140
11099-141
11099-142
11099-143
11099-144
11099-145
11099-146
11099-148
11099-15
11099-150
11099-152
11099-154
11099-155
11099-156
11099-157
11099-158
11099-159
11099-16
11099-160
11099-161
11099-162
11099-163
11099-164
11099-165
11099-166
11099-167
11099-168
11099-169
11099-17
11099-170
11099-171
11099-172
11099-173
11099-174
1109

11679-3
11679-4
11679-5
11679-6
11679-7
11679-8
11679-9
11684-1
11684-16
11684-17
11684-18
11684-19
11684-21
11684-23
11684-24
11684-25
11684-26
11684-28
11684-29
11684-34
11684-36
11684-38
11684-39
11684-40
11684-41
11684-42
11684-43
11684-44
11684-45
11684-46
11684-47
11684-6
11684-7
11684-8
11698-1
11698-10
11698-11
11698-12
11698-2
11698-3
11698-4
11698-5
11698-6
11698-7
11698-8
11698-9
11703-1
11703-10
11703-11
11703-12
11703-13
11703-14
11703-15
11703-2
11703-3
11703-4
11703-5
11703-7
11703-9
11710-10
11710-3
11763-1
11763-10
11763-11
11763-12
11763-13
11763-14
11763-16
11763-17
11763-18
11763-19
11763-2
11763-20
11763-22
11763-3
11763-4
11763-5
11763-6
11763-7
11763-8
11763-9
11764-1
11764-10
11764-12
11764-13
11764-14
11764-15
11764-16
11764-17
11764-18
11764-19
11764-2
11764-20
11764-21
11764-22
11764-23
11764-24
11764-25
11764-26
11764-27
11764-28
11764-29
11764-3
11764-30
11764-31
11764-32
11764-33
11764-34
11764-35
11764-36
11764-37
11764-38
11764-39
11764-4
11764-41
11764-

1222-160
1222-161
1222-162
1222-163
1222-164
1222-165
1222-168
1222-169
1222-17
1222-170
1222-173
1222-174
1222-176
1222-177
1222-178
1222-180
1222-181
1222-182
1222-183
1222-184
1222-185
1222-186
1222-187
1222-188
1222-189
1222-190
1222-191
1222-192
1222-193
1222-195
1222-196
1222-197
1222-198
1222-199
1222-2
1222-200
1222-202
1222-204
1222-205
1222-206
1222-207
1222-208
1222-209
1222-210
1222-211
1222-213
1222-214
1222-215
1222-216
1222-217
1222-218
1222-219
1222-22
1222-221
1222-222
1222-223
1222-224
1222-225
1222-226
1222-227
1222-228
1222-229
1222-23
1222-230
1222-231
1222-232
1222-233
1222-234
1222-236
1222-237
1222-238
1222-239
1222-240
1222-241
1222-242
1222-243
1222-244
1222-245
1222-247
1222-26
1222-27
1222-28
1222-29
1222-30
1222-32
1222-33
1222-335
1222-336
1222-337
1222-338
1222-339
1222-34
1222-340
1222-341
1222-342
1222-343
1222-344
1222-345
1222-346
1222-347
1222-348
1222-349
1222-35
1222-350
1222-351
1222-352
1222-353
1222-354
1222-355
1222-356
1222-357
1222-358
1222-3

12586-25
12586-26
12586-27
12586-31
12586-32
12586-33
12586-35
12586-36
12586-37
12586-38
12586-39
12586-4
12586-40
12586-41
12586-42
12586-43
12586-44
12586-46
12586-5
12586-54
12586-56
12586-58
12586-59
12586-6
12586-61
12586-63
12586-64
12586-66
12586-67
12586-68
12586-69
12586-7
12586-70
12586-72
12586-75
12586-8
12586-9
12663-1
12663-10
12663-2
12663-3
12663-4
12663-5
12663-6
12663-7
12663-8
12663-9
12673-1
12673-10
12673-11
12673-12
12673-14
12673-2
12673-4
12673-5
12673-6
12673-7
12673-8
12673-9
12679-1
12679-11
12679-12
12679-14
12679-15
12679-16
12679-2
12679-3
12679-5
12679-7
12679-8
12679-9
12704-2
12704-3
12727-10
12727-100
12727-101
12727-102
12727-103
12727-104
12727-105
12727-106
12727-108
12727-109
12727-11
12727-110
12727-113
12727-114
12727-115
12727-116
12727-117
12727-118
12727-119
12727-12
12727-120
12727-121
12727-122
12727-123
12727-125
12727-128
12727-129
12727-13
12727-130
12727-132
12727-133
12727-134
12727-135
12727-136
12727-137
12727-138
12727-139
12727-145

...

7224-78
7224-79
7224-8
7224-80
7224-81
7224-82
7224-85
7224-86
7224-87
7224-88
7224-89
7224-9
7224-90
7224-91
7224-92
7224-93
7224-94
7224-95
7224-96
7224-97
7224-98
7224-99
7225-1
7225-10
7225-11
7225-12
7225-2
7225-3
7225-31
7225-32
7225-4
7225-5
7225-6
7225-7
7289-1
7347-1
7347-2
7381-1
7381-100
7381-101
7381-109
7381-11
7381-112
7381-113
7381-114
7381-115
7381-117
7381-118
7381-119
7381-12
7381-120
7381-121
7381-122
7381-123
7381-124
7381-125
7381-126
7381-127
7381-128
7381-13
7381-130
7381-131
7381-132
7381-133
7381-134
7381-139
7381-14
7381-140
7381-141
7381-142
7381-143
7381-144
7381-145
7381-146
7381-147
7381-148
7381-149
7381-15
7381-150
7381-151
7381-152
7381-153
7381-154
7381-155
7381-156
7381-157
7381-158
7381-159
7381-160
7381-162
7381-163
7381-165
7381-166
7381-167
7381-168
7381-169
7381-170
7381-171
7381-172
7381-174
7381-175
7381-176
7381-177
7381-178
7381-179
7381-180
7381-181
7381-182
7381-183
7381-184
7381-185
7381-186
7381-187
7381-188
7381-189
7381-20
7381-21
7381-

7705-17
7705-2
7705-20
7705-21
7705-22
7705-23
7705-24
7705-25
7705-26
7705-29
7705-3
7705-30
7705-31
7705-32
7705-33
7705-34
7705-35
7705-36
7705-37
7705-38
7705-39
7705-4
7705-40
7705-41
7705-43
7705-5
7705-6
7705-7
7705-9
772-1
772-2
772-3
772-63
772-64
772-66
772-67
772-68
772-69
772-70
772-71
772-72
772-73
772-74
772-75
772-76
772-78
772-80
772-81
772-82
772-83
772-84
772-85
772-86
772-87
772-88
772-89
772-90
772-91
7760-1
7760-10
7760-11
7760-12
7760-13
7760-3
7760-4
7760-5
7760-6
7760-7
7760-8
7760-9
7809-9
7849-1
7849-11
7849-12
7849-13
7849-14
7849-15
7849-16
7849-17
7849-19
7849-20
7849-21
7849-22
7849-23
7849-24
7849-241
7849-242
7849-244
7849-247
7849-249
7849-25
7849-251
7849-26
7849-27
7849-28
7849-29
7849-4
7849-7
7849-8
7849-9
7860-10
7860-12
7860-13
7860-14
7860-15
7860-16
7860-17
7860-18
7860-19
7860-20
7860-21
7860-22
7860-23
7860-24
7860-26
7860-27
7860-28
7860-29
7860-30
7860-31
7860-32
7860-33
7860-34
7860-35
7860-36
7860-37
7860-38
7860-39
7860-40
7860-41
7860-42

865-63
865-64
865-7
865-8
865-9
8723-1
8723-10
8723-11
8723-12
8723-3
8723-4
8723-5
8723-6
8723-7
8723-8
8723-9
8779-1
8779-10
8779-11
8779-12
8779-13
8779-14
8779-16
8779-17
8779-18
8779-19
8779-2
8779-20
8779-21
8779-23
8779-24
8779-25
8779-26
8779-27
8779-3
8779-4
8779-5
8779-6
8779-7
8779-9
8822-10
8822-100
8822-101
8822-102
8822-104
8822-105
8822-106
8822-108
8822-11
8822-110
8822-111
8822-15
8822-16
8822-17
8822-18
8822-19
8822-20
8822-21
8822-22
8822-23
8822-24
8822-27
8822-29
8822-3
8822-31
8822-32
8822-33
8822-34
8822-35
8822-36
8822-37
8822-38
8822-39
8822-4
8822-41
8822-42
8822-43
8822-44
8822-46
8822-48
8822-49
8822-5
8822-50
8822-51
8822-52
8822-55
8822-56
8822-57
8822-58
8822-59
8822-6
8822-60
8822-61
8822-62
8822-63
8822-66
8822-67
8822-68
8822-69
8822-7
8822-70
8822-71
8822-73
8822-74
8822-75
8822-78
8822-79
8822-8
8822-81
8822-82
8822-83
8822-84
8822-85
8822-86
8822-87
8822-90
8822-91
8822-92
8822-93
8822-94
8822-95
8822-96
8822-98
8822-99
8832-10
8832-102
8832-103
883

9178-99
9233-10
9233-11
9233-12
9233-15
9233-16
9233-17
9233-18
9233-19
9233-20
9233-21
9233-22
9233-23
9233-24
9233-25
9233-26
9233-27
9233-28
9233-29
9233-30
9233-31
9233-33
9233-35
9233-36
9233-37
9233-38
9233-39
9233-41
9233-42
9233-43
9233-44
9233-45
9233-46
9233-47
9233-48
9233-49
9233-50
9233-51
9233-53
9233-54
9233-55
9233-56
9233-57
9233-58
9233-59
9233-6
9233-60
9233-61
9233-62
9233-63
9233-64
9233-65
9233-67
9233-68
9233-69
9249-100
9249-101
9249-102
9249-11
9249-12
9249-13
9249-14
9249-15
9249-16
9249-18
9249-24
9249-28
9249-29
9249-3
9249-30
9249-31
9249-32
9249-33
9249-34
9249-35
9249-36
9249-38
9249-39
9249-41
9249-46
9249-47
9249-48
9249-49
9249-5
9249-50
9249-51
9249-52
9249-54
9249-55
9249-57
9249-58
9249-59
9249-6
9249-60
9249-61
9249-62
9249-63
9249-64
9249-65
9249-66
9249-67
9249-68
9249-69
9249-70
9249-71
9249-73
9249-74
9249-75
9249-76
9249-77
9249-78
9249-79
9249-80
9249-81
9249-82
9249-83
9249-84
9249-85
9249-86
9249-89
9249-9
9249-90
9249-91
9249-92
9249-93
92

9678-9
9685-10
9685-11
9685-12
9685-13
9685-14
9685-2
9685-3
9685-4
9685-5
9685-6
9685-7
9685-8
9685-9
9714-1
9714-10
9714-11
9714-12
9714-14
9714-15
9714-16
9714-17
9714-18
9714-19
9714-2
9714-20
9714-21
9714-23
9714-27
9714-28
9714-29
9714-3
9714-30
9714-31
9714-33
9714-34
9714-35
9714-37
9714-38
9714-39
9714-4
9714-40
9714-41
9714-42
9714-43
9714-44
9714-45
9714-48
9714-49
9714-5
9714-50
9714-51
9714-53
9714-54
9714-55
9714-56
9714-57
9714-58
9714-59
9714-6
9714-60
9714-61
9714-62
9714-64
9714-65
9714-66
9714-67
9714-68
9714-7
9714-70
9714-71
9714-72
9714-73
9714-74
9714-8
9714-9
981-1
9950-10
9950-11
9950-2
9950-3
9950-30
9950-5
9950-6
9950-7
9950-8
9962-10
9962-11
9962-2
9962-4
9962-5
9962-6
9962-7
9962-8
9962-9
9971-1
9971-10
9971-100
9971-101
9971-102
9971-104
9971-105
9971-106
9971-107
9971-108
9971-109
9971-11
9971-110
9971-111
9971-113
9971-12
9971-13
9971-14
9971-15
9971-16
9971-17
9971-18
9971-19
9971-2
9971-20
9971-21
9971-22
9971-24
9971-25
9971-26
9971-27
9971-28
9971-3


In [103]:
obs_df = pd.concat(list_obs)
obs_df.head()

,subject_id,gender,age,sapsi_first,sofa_first,care_unit,amine,sedation,ventilation,period,series,y_event,y_mean_abpm
0,10013,F,87,15,8,1,1,0,0,10,"[[85.0, 89.1, 106.8, 31.5, 52.2, 1.0], [84.9, ...",1,62.465
1,10013,F,87,15,8,1,1,0,0,11,"[[85.9, 88.3, 123.2, 38.6, 63.2, 1.0], [85.5, ...",1,56.505
2,10013,F,87,15,8,1,1,0,0,12,"[[85.6, 93.6, 108.4, 34.7, 55.9, 1.0], [86.0, ...",1,60.52
3,10013,F,87,15,8,1,1,0,0,15,"[[86.9, 0.0, 95.3, 30.0, 48.5, 1.0], [87.4, 0....",1,55.23
4,10013,F,87,15,8,1,1,0,0,16,"[[87.3, 90.6, 105.9, 36.1, 55.9, 1.0], [86.8, ...",1,57.965


In [104]:
obs_df.shape

(39588, 13)

In [106]:
with open(PATH/'obs_df.pickle', 'wb') as f:
    pickle.dump(obs_df, f)

In [25]:
with open(PATH/'obs_df.pickle', 'rb') as f:
    obs_df = pickle.load(f)

In [26]:
obs_df.shape

(39588, 13)

In [27]:
np.random.seed(3)
patients_ids = obs_df.subject_id.unique()
is_val = np.random.choice(2, len(patients_ids ), p=[0.8, 0.2])

In [28]:
train_val_split = pd.DataFrame({"subject_id": patients_ids, "is_val": is_val})
train_val_split.head()

,subject_id,is_val
0,10013,0
1,1006,0
2,10061,0
3,10205,0
4,10209,1


In [29]:
obs_df_with_split = obs_df.merge(train_val_split, left_on="subject_id", right_on="subject_id")
obs_df_with_split.head()

,subject_id,gender,age,sapsi_first,sofa_first,care_unit,amine,sedation,ventilation,period,series,y_event,y_mean_abpm,is_val
0,10013,F,87,15,8,1,1,0,0,10,"[[85.0, 89.1, 106.8, 31.5, 52.2, 1.0], [84.9, ...",1,62.465,0
1,10013,F,87,15,8,1,1,0,0,11,"[[85.9, 88.3, 123.2, 38.6, 63.2, 1.0], [85.5, ...",1,56.505,0
2,10013,F,87,15,8,1,1,0,0,12,"[[85.6, 93.6, 108.4, 34.7, 55.9, 1.0], [86.0, ...",1,60.52,0
3,10013,F,87,15,8,1,1,0,0,15,"[[86.9, 0.0, 95.3, 30.0, 48.5, 1.0], [87.4, 0....",1,55.23,0
4,10013,F,87,15,8,1,1,0,0,16,"[[87.3, 90.6, 105.9, 36.1, 55.9, 1.0], [86.8, ...",1,57.965,0


In [30]:
obs_df_with_split.shape

(39588, 14)

In [31]:
obs_df_with_split["gender"] = obs_df_with_split["gender"].apply(lambda x: 1 if x == "F" else 0)

In [40]:
care2id = {v:k for k,v in enumerate(np.unique(obs_df_with_split.care_unit.values))}

In [41]:
care2id

{1: 0, 4: 1, 6: 2}

In [42]:
obs_df_with_split["care_unit"] = obs_df_with_split["care_unit"].apply(lambda x: care2id[x])

In [45]:
obs_df_with_split.tail()

,subject_id,gender,age,sapsi_first,sofa_first,care_unit,amine,sedation,ventilation,period,series,y_event,y_mean_abpm,is_val
39583,9991,1,70,23,9,1,0,0,0,74,"[[68.3, 97.1, 0.0, 0.0, 194.1, 0.0], [68.2, 97...",0,113.145,0
39584,9991,1,70,23,9,1,0,0,0,75,"[[78.2, 95.7, 173.4, 77.8, 115.4, 0.0], [79.7,...",0,116.635,0
39585,9991,1,70,23,9,1,0,0,0,77,"[[71.0, 94.0, 146.0, 61.0, 92.9, 0.0], [70.9, ...",0,103.56,0
39586,9991,1,70,23,9,1,0,1,0,8,"[[95.0, 98.0, 113.4, 48.4, 67.8, 0.0], [95.0, ...",0,68.41,0
39587,9991,1,70,23,9,1,0,1,0,9,"[[86.0, 98.0, 104.7, 51.9, 67.1, 0.0], [86.1, ...",0,69.69,0


In [46]:
train = obs_df_with_split.loc[obs_df_with_split["is_val"] == 0]
train.head()

,subject_id,gender,age,sapsi_first,sofa_first,care_unit,amine,sedation,ventilation,period,series,y_event,y_mean_abpm,is_val
0,10013,1,87,15,8,0,1,0,0,10,"[[85.0, 89.1, 106.8, 31.5, 52.2, 1.0], [84.9, ...",1,62.465,0
1,10013,1,87,15,8,0,1,0,0,11,"[[85.9, 88.3, 123.2, 38.6, 63.2, 1.0], [85.5, ...",1,56.505,0
2,10013,1,87,15,8,0,1,0,0,12,"[[85.6, 93.6, 108.4, 34.7, 55.9, 1.0], [86.0, ...",1,60.52,0
3,10013,1,87,15,8,0,1,0,0,15,"[[86.9, 0.0, 95.3, 30.0, 48.5, 1.0], [87.4, 0....",1,55.23,0
4,10013,1,87,15,8,0,1,0,0,16,"[[87.3, 90.6, 105.9, 36.1, 55.9, 1.0], [86.8, ...",1,57.965,0


In [47]:
valid = obs_df_with_split.loc[obs_df_with_split["is_val"] == 1]
valid.head()

,subject_id,gender,age,sapsi_first,sofa_first,care_unit,amine,sedation,ventilation,period,series,y_event,y_mean_abpm,is_val
221,10209,0,35,16,7,1,1,1,1,10,"[[101.1, 100.0, 149.3, 94.5, 115.7, 0.0], [101...",0,107.575,1
222,10209,0,35,16,7,1,1,1,1,102,"[[89.0, 100.0, 133.6, 69.2, 88.2, 0.0], [89.1,...",0,90.04,1
223,10209,0,35,16,7,1,1,1,1,103,"[[89.8, 97.9, 141.9, 71.2, 89.9, 0.0], [89.8, ...",0,96.515,1
224,10209,0,35,16,7,1,1,1,1,104,"[[89.4, 99.0, 159.3, 77.4, 98.1, 0.0], [89.1, ...",0,97.355,1
225,10209,0,35,16,7,1,1,1,1,106,"[[87.0, 99.0, 166.9, 78.4, 101.6, 0.0], [90.6,...",0,102.24,1


In [48]:
valid.shape

(7332, 14)

In [49]:
train.shape

(32256, 14)

In [50]:
with open(PATH/'obs_df_train.pickle', 'wb') as f:
    pickle.dump(train, f)

In [51]:
with open(PATH/'obs_df_valid.pickle', 'wb') as f:
    pickle.dump(valid, f)

In [52]:
np.sqrt(364)

19.078784028338912